<a href="https://colab.research.google.com/github/vijjus/CV/blob/master/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pretrainedmodels

     |████████████████████████████████| 61kB 4.0MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=4d64b56402c98675ce79dddfab97768f7dd76bb1955cf360a1379c7d79bba260
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [3]:

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
#import timm
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.ops as ops
import cv2
from collections import OrderedDict

plt.ion()   # interactive mode 

In [4]:
!echo '3FDsALdx4sLyMedGxO5h:RsdZX1E2jRryINc6Vb5HGpL1FY84lNBXbTNSU8Ln' > ~/.passwd-s3fs
!cat ~/.passwd-s3fs
!chmod 600 ~/.passwd-s3fs
!mkdir /s3
!apt install s3fs
!s3fs w251-final-project /s3 -o passwd_file=~/.passwd-s3fs,url=https://s3.us.cloud-object-storage.appdomain.cloud

3FDsALdx4sLyMedGxO5h:RsdZX1E2jRryINc6Vb5HGpL1FY84lNBXbTNSU8Ln
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  s3fs
0 upgraded, 1 newly installed, 0 to remove and 59 not upgraded.
Need to get 200 kB of archives.
After this operation, 557 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 s3fs amd64 1.82-1 [200 kB]
Fetched 200 kB in 0s (2,973 kB/s)
Selecting previously unselected package s3fs.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../archives/s3fs_1.82-1_amd64.deb ...
Unpacking s3fs (1.82-1) ...
Setting up s3fs (1.82-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [5]:
!ls /s3/DFDC

dataloader	   dfdc_train_part_1  dfdc_train_part_3  dfdc_train_part_5
dfdc_train_part_0  dfdc_train_part_2  dfdc_train_part_4  models


In [6]:
model_name = "xception"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

In [7]:
import pretrainedmodels
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [8]:
print(pretrainedmodels.pretrained_settings[model_name])

{'imagenet': {'url': 'http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth', 'input_space': 'RGB', 'input_size': [3, 299, 299], 'input_range': [0, 1], 'mean': [0.5, 0.5, 0.5], 'std': [0.5, 0.5, 0.5], 'num_classes': 1000, 'scale': 0.8975}}


In [9]:
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
#model.eval()

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth" to /root/.cache/torch/checkpoints/xception-43020ad28.pth


In [10]:
######################################################################
# Load Data
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        #transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #Noise(0.25, 0.05)
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        #transforms.CenterCrop(229),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [11]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
    # this is what ImageFolder normally returns 
        try:
            original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
            # the image file path
            path = self.imgs[index][0]

            # make a new tuple that includes original and the path
            tuple_with_path = (original_tuple + (path,))

            return tuple_with_path
        
        except Exception as e:
            return None

In [12]:
def my_collate(batch): # batch size 4 [{tensor image, tensor label},{},{},{}] could return something like G = [None, {},{},{}]
    batch = list(filter (lambda x:x is not None, batch)) # this gets rid of nones in batch. For example above it would result to G = [{},{},{}]
    # I want len(G) = 4
    # so how to sample another dataset entry?
    return torch.utils.data.dataloader.default_collate(batch) 

In [13]:
# import torch.utils.data
import random
import torch
is_torchvision_installed = True
try:
    import torchvision
except:
    is_torchvision_installed = False
class BalancedBatchSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset, labels=None):
        self.labels = labels
        self.dataset = dict()
        self.balanced_max = 0
        # Save all the indices for all the classes
        for idx in range(0, len(dataset)):
            label = self._get_label(dataset, idx)
            if label not in self.dataset:
                self.dataset[label] = list()
            self.dataset[label].append(idx)
            self.balanced_max = len(self.dataset[label]) \
                if len(self.dataset[label]) > self.balanced_max else self.balanced_max
        
        # Oversample the classes with fewer elements than the max
        for label in self.dataset:
            while len(self.dataset[label]) < self.balanced_max:
                self.dataset[label].append(random.choice(self.dataset[label]))
        self.keys = list(self.dataset.keys())
        self.currentkey = 0
        self.indices = [-1]*len(self.keys)

    def __iter__(self):
        while self.indices[self.currentkey] < self.balanced_max - 1:
            self.indices[self.currentkey] += 1
            yield self.dataset[self.keys[self.currentkey]][self.indices[self.currentkey]]
            self.currentkey = (self.currentkey + 1) % len(self.keys)
        self.indices = [-1]*len(self.keys)
    
    def _get_label(self, dataset, idx, labels = None):
        if self.labels is not None:
            return self.labels[idx].item()
        else:
            # Trying guessing
            dataset_type = type(dataset)
            if is_torchvision_installed and dataset_type is torchvision.datasets.MNIST:
                return dataset.train_labels[idx].item()
            elif is_torchvision_installed and dataset_type is torchvision.datasets.ImageFolder:
                return dataset.imgs[idx][1]
            elif is_torchvision_installed and dataset_type.__bases__[0] is torchvision.datasets.folder.ImageFolder:
                return dataset.imgs[idx][1]
            else:
                raise Exception("You should pass the tensor of labels to the constructor as second argument")

    def __len__(self):
        return self.balanced_max*len(self.keys)

In [14]:
%%time

data_dir = '/s3/DFDC/dataloader/'

image_datasets = {x: ImageFolderWithPaths(os.path.join(data_dir, x), data_transforms[x])                    
                  for x in ['train', 'val']}

# balanced sampler dataloader
dataloaders = {x: torch.utils.data.DataLoader(
    image_datasets[x], sampler=BalancedBatchSampler(image_datasets[x], None), 
    batch_size=16, shuffle=False, num_workers=2, collate_fn=my_collate)
               for x in ['train', 'val']}


dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

CPU times: user 210 ms, sys: 13.8 ms, total: 224 ms
Wall time: 5min 58s


In [15]:
image_datasets

{'train': Dataset ImageFolderWithPaths
     Number of datapoints: 29054
     Root location: /s3/DFDC/dataloader/train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(299, 299), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ), 'val': Dataset ImageFolderWithPaths
     Number of datapoints: 16421
     Root location: /s3/DFDC/dataloader/val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=PIL.Image.BILINEAR)
                CenterCrop(size=(229, 229))
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [16]:
class_names

['FAKE', 'REAL']

In [17]:
# visualize images
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [18]:
from random import randrange

In [19]:
def get_sample(s):
    ds = image_datasets[s]
    sz = len(ds)
    n = randrange(20)
    print(sz,n)
    i = 0
    for data in dataloaders[s]:
        i += 1
        if i == n:
            X, y, z = data
            return X, y, z

In [20]:
class MyNet(torch.nn.Module):
  def __init__(self, model_name):
    super(MyNet, self).__init__()
    self.net = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    self.fc = nn.Linear(self.net.last_linear.out_features, 2)

  def forward(self, x):
    x = self.net(x)
    return self.fc(x)

In [21]:
net = MyNet(model_name)

In [22]:
#X, y, z = get_sample('train')

In [23]:
def show_image(X):
    img = X/X.max().item()
    img = (img + 1)/2.0
    plt.imshow(img.permute(1,2,0).numpy())

In [24]:
from tqdm import tqdm_notebook

In [25]:
# function to train model
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            
            dataset_size = 0
            
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels, paths in tqdm.notebook.tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                dataset_size += len(paths)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                        
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size         

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            print(dataset_size)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [26]:
# visualize predictions

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels, paths) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
#             paths = paths.to(device)
            # asynchronous
#             inputs = inputs.to(device, non_blocking=True)
#             labels = labels.to(device, non_blocking=True)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

## Train with seed 1337

In [27]:
#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False

In [28]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
set_seed_everywhere(1337, device)

In [30]:
# load model onto each GPU
net = net.to(device)

In [31]:
# asynchronous
# model_ft = model_ft.to(device, non_blocking=True) 

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(net.parameters(), lr=0.001)

# # 0.1 decay rate every 7 steps
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [33]:
import tqdm

In [ ]:
# train and evaluate model
model_ft = train_model(net, criterion, optimizer_ft,
                         exp_lr_scheduler, num_epochs=10)

visualize_model(model_ft)

plt.ioff()
plt.show()

Epoch 0/9
----------
